<a href="https://colab.research.google.com/github/Benteaux/karpathy-tutorials/blob/main/notebooks/makemore1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
names = open("names.txt", "r").read().splitlines()

In [ ]:
len(names)

32033

In [ ]:
max(len(w) for w in names)

15

In [ ]:
import torch

In [ ]:
N = torch.zeros([27, 27])

In [ ]:
alphabet = sorted(list(set(''.join(names))))

In [ ]:
stoi = {s:i+1 for i, s in enumerate(alphabet)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [ ]:
stoi

In [ ]:
count = 0
for w in names:
  chs = ['.'] + list(w) + ['.']
  for char1, char2 in zip(chs, chs[1:]): # zip just stops when it runs out of corresponding items to iterate over
    index1 = stoi[char1]
    index2 = stoi[char2]
    N[index1, index2] += 1
    count += 1


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (100, 100))
plt.imshow(N, cmap = "Pastel2_r")
for i in range(27):
  for j in range(27):
    chstr = itos[i] + itos[j]
    plt.text(j, i, chstr, ha = "center", va = "bottom", color = "black")
    plt.text(j, i, N[i][j].item(), ha = "center", va = "top", color = "black")

plt.axis('off')

(-0.5, 26.5, 26.5, -0.5)

In [ ]:
P = (N + 1).float()
P /= P.sum(1, keepdim = True)

In [ ]:
N.shape

torch.Size([27, 27])

In [ ]:
P.shape

torch.Size([27, 27])

In [ ]:
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
  theName = []
  i = 0
  while True:
    p = P[i]
    i = torch.multinomial(p, num_samples = 1, replacement = True, generator = g).item()
    if i == 0:
      break
    theName.append(itos[i])

  print(''.join(theName))

junide
janasah
p
cony
a
nn
kohin
tolian
juee
ksahnaauranilevias


In [ ]:
# for every bigram in the dataset, calculate the log likelihood, then negate it, then average it
n = 0
log_likelihood = 0
for word in names:
  chrs = '.' + str(word) + '.'
  for ch1, ch2 in zip(chrs, chrs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1][ix2]
    log_likelihood += torch.log(prob)
    n += 1
    # print(f'{ch1}{ch2}: prob of {prob}')

print(f'{log_likelihood=}')
print(f'{-log_likelihood=}')
nlla = -log_likelihood / n
print(f'{nlla=}')

log_likelihood=tensor(-559951.5625)
-log_likelihood=tensor(559951.5625)
nlla=tensor(2.4544)


In [ ]:
# data generation
xs = []
ys = []
for word in names:
  chrs = '.' + str(word) + '.'
  for ch1, ch2 in zip(chrs, chrs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
otherN = torch.zeros([228146, 27])

In [ ]:
for row, column in zip(xs, ys):
  otherN[row, column] += 1

In [ ]:
otherN

tensor([[   0., 4410., 1306.,  ...,  134.,  535.,  929.],
        [6640.,  556.,  541.,  ...,  182., 2050.,  435.],
        [ 114.,  321.,   38.,  ...,    0.,   83.,    0.],
        ...,
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.]])

In [ ]:
trueProbs = otherN + 1
trueProbs /= trueProbs.sum(1, keepdims = True)

In [ ]:
# setting up the network
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator = g, requires_grad = True)

In [ ]:
import torch.nn.functional as F

In [ ]:
# for index in xs, index the corresponding row of W. Then, sum the columns of W, keeping the dimensions so we have [len(ix) x 1]

In [ ]:
# note that for Exercise 6, I decided to try out KL divergence loss. That's why I make the whole "otherN" and "trueProbs" tensors
# i have yet to figure out why the loss star
# forward pass
for i in range(50):
  logits = W[xs]
  #loss = F.cross_entropy(logits, ys, reduction = "mean")

  counts = logits.exp()
  targetProbs = counts / counts.sum(1, keepdims = True)

    # Compute the KL divergence
  loss = F.kl_div(targetProbs.log(), trueProbs)


  #loss = targetProbs[xs, ys].log().mean()

  # backward pass
  W.grad = None
  loss.backward()
  print(loss.item())

  # optimization
  W.data += -0.0001 * W.grad
print(loss.item())

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2943: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


tensor([[0.0607, 0.0100, 0.0123,  ..., 0.0118, 0.1537, 0.1459],
        [0.0290, 0.0796, 0.0248,  ..., 0.0118, 0.0022, 0.0472],
        [0.0312, 0.0737, 0.0484,  ..., 0.1204, 0.0469, 0.0126],
        ...,
        [0.0301, 0.0080, 0.0090,  ..., 0.0125, 0.0531, 0.0111],
        [0.0634, 0.0270, 0.0101,  ..., 0.0833, 0.0175, 0.0517],
        [0.0308, 0.0113, 0.0178,  ..., 0.0190, 0.0116, 0.0562]],
       grad_fn=<DivBackward0>)
tensor([[3.1192e-05, 1.3759e-01, 4.0767e-02,  ..., 4.2109e-03, 1.6719e-02,
         2.9008e-02],
        [1.9583e-01, 1.6425e-02, 1.5983e-02,  ..., 5.3963e-03, 6.0480e-02,
         1.2857e-02],
        [4.3039e-02, 1.2051e-01, 1.4596e-02,  ..., 3.7425e-04, 3.1437e-02,
         3.7425e-04],
        ...,
        [3.7037e-02, 3.7037e-02, 3.7037e-02,  ..., 3.7037e-02, 3.7037e-02,
         3.7037e-02],
        [3.7037e-02, 3.7037e-02, 3.7037e-02,  ..., 3.7037e-02, 3.7037e-02,
         3.7037e-02],
        [3.7037e-02, 3.7037e-02, 3.7037e-02,  ..., 3.7037e-02, 3.7037e-02

KeyboardInterrupt: ignored

In [ ]:
g = torch.Generator().manual_seed(2147483647)

# sampling
# start w/ start token, calculating logits, then probs, then multinomial sample, spit out result
ix = 0
for i in range(5):
  ix = 0
  out = []
  while True:
    xenc = F.one_hot(torch.tensor([ix]), num_classes = 27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims = True)
    ix = torch.multinomial(probs, num_samples = 1, generator = g, replacement = True)
    out.append(itos[ix.item()])
    if ix == 0:
      break
  print(''.join(out))


ouwjdjdjawccadjufhqyywednw.
.
oiinwtoziaszwtwt.
ydaxjiauydbbleviajsdbduinrwipblvszwygnbyt.
otblufrmumkhyfodtumjmpgniszwjqrzafqyhogdaygezocckkpjabwtwtdkmriibwyfinwtgcasnhsvfbmofbuxhddgogfbptpasilu.


In [ ]:
# W is 27 x 27
# xs is a list of all the indices we need weights for
# those weights are accessed via isolating the relevant column?

**E02: Split Dataset**

In [ ]:
import random

In [ ]:
trainIndex = (int) (len(names) * 0.8)
testIndex = trainIndex + (int) ((len(names) - trainIndex) / 2)
print(names[0])
random.shuffle(names)
print(names[0])

baron
rizzo


In [ ]:
trainX, trainY, testX, testY, devX, devY = [], [], [], [], [], []
for name in names[:trainIndex]:
  name = ['.'] + list(name) + ['.']
  for ch1, ch2 in zip(name, name[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    trainX.append(ix1)
    trainY.append(ix2)

for name in names[trainIndex:testIndex]:
  name = ['.'] + list(name) + ['.']
  for ch1, ch2 in zip(name, name[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    testX.append(ix1)
    testY.append(ix2)

for name in names[testIndex:]:
  name = ['.'] + list(name) + ['.']
  for ch1, ch2 in zip(name, name[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    devX.append(ix1)
    devY.append(ix2)


In [ ]:
trainX = torch.tensor(trainX)
trainY = torch.tensor(trainY)
testX = torch.tensor(testX)
testY = torch.tensor(testY)
devX = torch.tensor(devX)
devY = torch.tensor(devY)

In [ ]:
# setting up the network
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator = g, requires_grad = True)

In [ ]:
# first part of training

# forward pass
for i in range(25):
  xenc = F.one_hot(trainX, num_classes = 27).float()
  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims = True)
  loss = -probs[trainX, trainY].log().mean() + 0.1 * (W**2).mean()

  # backward pass
  W.grad = None
  loss.backward()
  #print(loss.item())

  # optimization
  W.data += -50 * W.grad
print(loss.item())
# 2.721 w/o regularization
# 2.762 w/ 0.01 regularization

2.794311046600342


In [ ]:
# dev testing
xenc = F.one_hot(devX, num_classes = 27).float()
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims = True)
loss = -probs[devX, devY].log().mean()
print(loss.item())
# 3.1745 w/o regularization
# 3.16 w/ 0.01 regularization

3.1065051555633545


In [ ]:
# second part of training
for i in range(25):
  xenc = F.one_hot(trainX, num_classes = 27).float()
  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims = True)
  loss = -probs[trainX, trainY].log().mean() + 0.1 * (W**2).mean()

  # backward pass
  W.grad = None
  loss.backward()
  W.data += -50 * W.grad
print(loss.item())
# 2.6806 w/o regularization
# 2.6783 w/ 0.01 regularization

2.7128818035125732


In [ ]:
# test testing

In [ ]:
xenc = F.one_hot(testX, num_classes = 27).float()
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims = True)
loss = -probs[testX, testY].log().mean()
print(loss.item())
# 3.068 w/o regularization
# 3.054 w/ 0.01 regularization

2.989832878112793


In [ ]:
# the model has overfit. still improved after dev, but overfit nonetheless